## Introduction

#### create a model to predict solar energy efficiency based on the measurements of various meteorological parameters over a period of time.

### Import the necessary packages.

In [2]:
!pip install imbalanced-learn==0.7.0 -q
!pip install --user pycaret[full] -q
!pip install numba==0.53 -q

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t

In [3]:
!pip install metpy -q

In [4]:
!pip install pvlib -q

In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # visualization
from matplotlib import pyplot as plt # visualization
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import missingno as msno
import re
import copy

from scipy.stats import skew, kurtosis
from prettytable import PrettyTable
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
import metpy.calc as mpcalc
import tqdm
from tqdm.notebook import  tqdm_notebook
from tqdm.autonotebook import tqdm
tqdm.pandas()

from sklearn.model_selection import  TimeSeriesSplit,cross_val_score,KFold,train_test_split,ShuffleSplit,StratifiedKFold,learning_curve,RandomizedSearchCV
from catboost import CatBoostClassifier,Pool,cv,monoforest,CatBoostRegressor
import optuna
from optuna.samplers import RandomSampler,TPESampler,MOTPESampler,CmaEsSampler
from sklearn.metrics import f1_score,classification_report,confusion_matrix,mean_squared_error,r2_score
from xgboost import XGBClassifier,plot_tree,XGBRegressor
import xgboost as xgb
from optuna.integration import XGBoostPruningCallback,LightGBMPruningCallback
from sklearn.preprocessing import LabelEncoder,StandardScaler,PolynomialFeatures
import lightgbm as lgbm
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import FunctionTransformer,SplineTransformer
from sklearn.ensemble import StackingRegressor,RandomForestRegressor
from pvlib import location
import pvlib

from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

### Load train and test data

In [11]:
train=pd.read_csv("/kaggle/input/mh-forecasting-solar-energy-efficiency/train.csv")
test=pd.read_csv("/kaggle/input/mh-forecasting-solar-energy-efficiency/test.csv")

### Function to pre-process the data

In [12]:
def pre_process(df,data=None):
    
    def remove_col_name_space(df):
        df=df.rename(columns={col:re.sub("\s","_",col.lower()) for col in df.columns.values})
        return df
    
    df=remove_col_name_space(df)
    
    def sin_transformer(period):
        return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))


    def cos_transformer(period):
        return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))
    
    
    def periodic_spline_transformer(period, n_splines=None, degree=3):
        if n_splines is None:
            n_splines = period
        n_knots = n_splines + 1  # periodic and include_bias is True
        return SplineTransformer(
                        degree=degree,
                        n_knots=n_knots,
                        knots=np.linspace(0, period, n_knots).reshape(n_knots, 1),
                        extrapolation="periodic",
                        include_bias=True)

    
    
    
    df['date_time']=pd.to_datetime(df["timestamp"]) #convert to date-time format
    df['date']=df['date_time'].dt.date
    df['day']=df['date_time'].dt.day #extract day from the date
    df['day_label']=df['date_time'].dt.day_name() #extract the day name from the date
    df['day_number']=df['date_time'].dt.dayofweek #extract the day number from the date
    df['month_number']=df['date_time'].dt.month #extract month number from the date
    df['month_label']=df['date_time'].dt.strftime('%b') #extract the month name from the date
    df['year_quarter']=df['date_time'].dt.quarter #extract the quarter of the year
    df['week_of_year']=df['date_time'].dt.week #extract week of the year from date
    df['year']= df['date_time'].dt.year #extract year
    df['dayofmonth'] = df['date_time'].dt.daysinmonth #extract the day of the month
    df['dayofyear'] = df['date_time'].dt.day_of_year #extract day of the year
    df['weekday']=df['date_time'].dt.day_name().isin(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']).astype('int') #create weekday column
    df['weekend']=df['date_time'].dt.day_name().isin(['Saturday', 'Sunday']).astype('int') #create weekend column
    df['month_start']=df['date_time'].dt.is_month_start.astype('int') #create month start
    df['month_end']=df['date_time'].dt.is_month_end.astype('int') #create month end
    df['quarter_start']=df['date_time'].dt.is_quarter_start.astype('int') #create quarter start
    df['quarter_end']=df['date_time'].dt.is_quarter_end.astype('int') #create quarter end
    df['year_start']=df['date_time'].dt.is_year_start.astype('int') #create year start
    df['year_end']=df['date_time'].dt.is_year_end.astype('int') #create year end
    df['hour']=df['date_time'].dt.hour
    df['minute']=df['date_time'].dt.minute
    
    def cycle_feat(df):
        for col,period in zip(['hour','month_number','week_of_year',
                           'day_number','day','dayofyear',
                           'year_quarter'],[24,12,52,7,31,365,4]):
            df[f"{col}_sin"] = sin_transformer(period).fit_transform(df[[col]])
            df[f"{col}_cos"] = cos_transformer(period).fit_transform(df[[col]])
        return df
    
    
    
    def spline_feat(df):
        for col,period,spline in zip(['hour','month_number','week_of_year',
                                      'day_number','day','dayofyear',
                                      'year_quarter'],
                                     [24,12,52,7,31,365,4],
                                     [12,6,26,3,15,182,4]):
            spline_df=pd.DataFrame(periodic_spline_transformer(period=period, n_splines=spline).fit_transform(df[[col]]),
                                      columns=[f"{col}_spline_{n}" for n in range(spline)])
            df=pd.concat([df,spline_df],axis=1)
        return df
    
    
    
    #df=spline_feat(df)
    df=cycle_feat(df)
   
    
    #function to convert month to seasons
    def month2seasons(x):
        if x in [6,7,8,9]:
            season = 'Monsoon'
        elif x in [3,4,5]:
            season = 'Summer'
        elif x in [10,11]:
            season = 'Post-monsoon'
        elif x in [1,2,12]:
            season = 'Winter'
        return season
       

 
    df['seasons']=df['month_number'].apply(month2seasons)

    def hours2timing(x):
        if x in range(20,23):
            timing = 'Night'
        elif x in range(5,12):
            timing = 'Morning'
        elif x in range(12, 16):
            timing = 'Afternoon'
        elif x in range(16, 20):
            timing = 'Evening'
        elif x in [23,0,1,2,3,4]:
            timing = 'Midnight'    
        else:
            timing = 'X'
        return timing
    
    df['timings']=df['hour'].apply(hours2timing)

    
    df['solar_zenith_angle_radians']=np.cos(np.radians(df['solar_zenith_angle']))
    df['wind_direction_to_radians']=np.deg2rad(df['wind_direction'])

    df['wind_vector_u']=df.apply(lambda x:x['wind_speed']*np.cos(x['wind_direction']),axis=1)
    df['wind_vector_v']=df.apply(lambda x:x['wind_speed']*np.sin(x['wind_direction']),axis=1)
    df['cardinal_direction']=mpcalc.angle_to_direction(df['wind_direction'], full=True)
    
    #df['clearness_index']= 0.3276+(1.4194*df['solar_zenith_angle_radians'])-(1.78262*df['solar_zenith_angle_radians']**2)+(0.836565*df['solar_zenith_angle_radians']*3) 
    #df['clear_period']=-0.8589+(3.6578*df['clearness_index'])-(3.6220*df['clearness_index']**2) +(1.9620*df['clearness_index']**3)
    
    
    
    df['temp_dew'] = df['temperature']-((100-df['relative_humidity'])/5)
    df['altitude']=pvlib.atmosphere.pres2alt(df['pressure'])
    df['site_pressure']=pvlib.atmosphere.alt2pres(df['altitude'])
    df['airmass_relative']=location.atmosphere.get_relative_airmass(df['solar_zenith_angle'])
    df['airmass_abs']=pvlib.atmosphere.get_absolute_airmass(df['airmass_relative'],df['pressure'])
    
    #for mod_type in ['cdte','monosi','xsi','multisi','polysi','cigs','asi']:
    df[f'spectral_mismatch_cigs']=list(pvlib.atmosphere.first_solar_spectral_correction(df['precipitable_water']/10,df['airmass_abs'],
                                                                                                  min_pw=min(df['precipitable_water']/10),
                                                                                                  max_pw=max(df['precipitable_water']/10),
                                                                                                  module_type='cigs'))
    
    
    
    df['ozone_concentration_(ppm)']=df['ozone']/1000
    df['ozone_layer_thickness_(du)']=sum(df['ozone'])/df['solar_zenith_angle_radians']
    
    
    num_col=['temperature', 'dew_point', 'surface_albedo', 'pressure',
             'wind_direction_to_radians', 'wind_speed', 'ozone','solar_zenith_angle_radians', 
             'precipitable_water', 'relative_humidity','temp_dew','altitude','site_pressure',
             'airmass_relative','airmass_abs','spectral_mismatch_cigs',
            'ozone_concentration_(ppm)','ozone_layer_thickness_(du)']
    
    cat_col=['cloud_type','date','hour','wind_direction_to_radians']
    
  
    def shift_feat(df):
        for col,shift_len in zip(['temperature', 'dew_point', 'wind_direction_to_radians',
                                  'wind_speed','solar_zenith_angle_radians', 
                                  'relative_humidity'],[24,6,4,10,30,13]):
            df[f"{col}_lag_{shift_len}"]=df[col].diff().fillna(0).shift(shift_len)
        return df
    
    df=shift_feat(df)
            

    
    """group by numerical summary of each numerical column"""
    def feature_eng(df):
        for valcol in num_col:
            df1=(df.groupby(cat_col)[valcol].agg({'min','median','mean','max',np.std}).reset_index())
            df1=(df1.rename(columns={c:valcol+'_'+c for c in df1.loc[:,df1.columns.str.
                                                match("(min|median|mean|max|std)")]}))
            df=pd.merge(df,df1,on=cat_col,how='left')
        return df  


    df=feature_eng(df)
    
    def window_feat(df,cols):
        for col in cols:
            df[f"{col}_win7_min"]=df[col].diff().fillna(0).rolling(window=12).min()
            df[f"{col}_win7_max"]=df[col].diff().fillna(0).rolling(window=12).max()
            df[f"{col}_win7_mean"]=df[col].diff().fillna(0).rolling(window=12).mean()
            df[f"{col}_win7_std"]=df[col].diff().fillna(0).rolling(window=12).std()
            df[f"{col}_win7_var"]=df[col].diff().fillna(0).rolling(window=12).var()
        return df

    df=window_feat(df,cols=['temperature', 'dew_point', 'wind_direction_to_radians',
                                  'wind_speed','solar_zenith_angle_radians', 
                                  'relative_humidity']) 

    def expand_feat(df,cols):
        for col in cols:
            df[f"{col}_exwin7_min"]=df[col].diff().fillna(0).expanding(12).min()
            df[f"{col}_exwin7_max"]=df[col].diff().fillna(0).expanding(12).max()
            df[f"{col}_exwin7_mean"]=df[col].diff().fillna(0).expanding(12).mean()
            df[f"{col}_exwin7_std"]=df[col].diff().fillna(0).expanding(12).std()
            df[f"{col}_exwin7_var"]=df[col].diff().fillna(0).expanding(12).var()
        return df

    df=expand_feat(df,cols=['temperature', 'dew_point', 'wind_direction_to_radians',
                                  'wind_speed','solar_zenith_angle_radians', 
                                  'relative_humidity']) 

    
    #z-score outlier detection for numerical columns  
    def outlier_z(df,x):
        threshold = 3
        out=[]
        mean = np.mean(df[x])
        std = np.std(df[x])
        for i in df[x]:
            if ((i-mean)/std) > threshold:
                out.append(1)
            else:
                out.append(0)
        return out
    
    """"
    for col in num_col:
        df[f"{col}_outlier"]= outlier_z(df,col)
    
    
    for col in num_col:
        df[f"{col}_skewness"]=df.groupby(cat_col)[col].transform(pd.Series.skew)
        df[f"{col}_kurtosis"]=df.groupby(cat_col)[col].transform(pd.Series.kurt)
    """
    
    
   
    if data is "train":
        df['diffuse_fraction']=round(df['clearsky_dhi']/df['clearsky_ghi'],3)          
    df=df.fillna(0)   
    return df
    

### Apply pre-process function to train data.

In [13]:
train=pre_process(train,data='train')

In [25]:
for i, col in enumerate(train.columns):
    print(f"{i}_{col}") # 1:5,6,11:13,14,15,25,28:36,37:163  [1,2,4,6,11,14,15,48,49]

0_timestamp
1_temperature
2_dew_point
3_surface_albedo
4_pressure
5_wind_direction
6_wind_speed
7_clearsky_dhi
8_clearsky_dni
9_clearsky_ghi
10_fill_flag
11_ozone
12_cloud_type
13_solar_zenith_angle
14_precipitable_water
15_relative_humidity
16_date_time
17_date
18_day
19_day_label
20_day_number
21_month_number
22_month_label
23_year_quarter
24_week_of_year
25_year
26_dayofmonth
27_dayofyear
28_weekday
29_weekend
30_month_start
31_month_end
32_quarter_start
33_quarter_end
34_year_start
35_year_end
36_hour
37_minute
38_hour_sin
39_hour_cos
40_month_number_sin
41_month_number_cos
42_week_of_year_sin
43_week_of_year_cos
44_day_number_sin
45_day_number_cos
46_day_sin
47_day_cos
48_dayofyear_sin
49_dayofyear_cos
50_year_quarter_sin
51_year_quarter_cos
52_seasons
53_timings
54_solar_zenith_angle_radians
55_wind_direction_to_radians
56_wind_vector_u
57_wind_vector_v
58_cardinal_direction
59_temp_dew
60_altitude
61_site_pressure
62_airmass_relative
63_airmass_abs
64_spectral_mismatch_cigs
65_o

In [27]:
X=train.iloc[:,np.r_[1:7,11:16,25,28:36,37:213]].copy()
y=train['clearsky_dhi']#,'clearsky_dni','clearsky_ghi']].copy()
#dhi=train['clearsky_dhi']
#dni=train['clearsky_dni']
#ghi=train['clearsky_ghi']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42,shuffle=False)

In [28]:
new_train=pd.concat([X_train,y_train])

In [28]:
train_1=pd.concat([X,y],axis=1)

### Train seperate for model each target label.

In [19]:
import pycaret
from pycaret.regression import *

In [29]:
reg_pycaret = setup(data = train_1,
                    data_split_shuffle=False,
                    fold_strategy="timeseries",
                    fold_shuffle=False,
                    fold=5,
                    target = 'clearsky_dhi', session_id=112,
                    use_gpu =True,
                    
                    ) 
set_config('seed', 123)

,Description,Value
0,Session id,112
1,Target,clearsky_dhi
2,Target type,Regression
3,Original data shape,"(210240, 197)"
4,Transformed data shape,"(210240, 219)"
5,Transformed train set shape,"(147168, 219)"
6,Transformed test set shape,"(63072, 219)"
7,Numeric features,193
8,Categorical features,3
9,Preprocess,True


In [30]:
dhi_top3 = compare_models(sort = 'MSE',n_select=3,
                     include=['catboost','lightgbm','xgboost'],
                     )

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,21.6619,1849.8036,40.3587,0.8680,0.7947,0.2490,17.2300
xgboost,Extreme Gradient Boosting,22.5897,1928.6858,41.8231,0.8643,0.8794,0.2531,3.1560


Processing:   0%|          | 0/19 [00:00<?, ?it/s]

In [23]:
dhi_blender = blend_models(dhi_top3)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,32.7600,3817.9875,61.7899,0.6953,0.8624,0.3825
1,13.6319,719.2966,26.8197,0.9464,0.5648,0.1550
2,20.7954,1611.1874,40.1396,0.8996,0.5567,0.2164
3,11.3896,563.5606,23.7394,0.9536,0.5154,0.1378
4,23.6835,1805.3409,42.4893,0.9001,0.6962,0.2624
Mean,20.4521,1703.4746,38.9956,0.8790,0.6391,0.2308
Std,7.6248,1162.5892,13.5211,0.0946,0.1271,0.0879


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [15]:
dhi_final = finalize_model(dhi_blender)

In [20]:
dni_top3 = compare_models(sort = 'MSE',n_select=3,
                     exclude=['rf','dt','ada','et','en','lasso','br','ridge','huber','gbr','par','lr','lar'],
                     )

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,44.3957,5189.8082,71.7317,0.9333,1.4945,0.4608,13.3300
lightgbm,Light Gradient Boosting Machine,42.1869,5274.5765,72.4456,0.9323,0.9262,0.4178,11.1660
xgboost,Extreme Gradient Boosting,49.5340,6365.5112,79.3283,0.9182,1.6339,0.5063,7.6520
knn,K Neighbors Regressor,49.7923,8066.4939,89.6790,0.8966,0.3915,0.5729,6.9300
llar,Lasso Least Angle Regression,253.7169,78352.7316,279.7853,-0.0052,3.9265,2.5821,8.6820
dummy,Dummy Regressor,253.7169,78352.7316,279.7853,-0.0052,3.9265,2.5821,5.1420
omp,Orthogonal Matching Pursuit,237.2836,3544682.9415,917.8382,-43.5733,2.7252,2.0538,7.4980


Processing:   0%|          | 0/35 [00:00<?, ?it/s]

In [21]:
dni_blender = blend_models(dni_top3)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,49.1833,5631.1019,75.0407,0.9292,1.8217,0.4706
1,39.6480,4879.3712,69.8525,0.9328,1.1376,0.2897
2,40.7385,4516.5437,67.2052,0.9473,1.3369,0.4545
3,35.7626,4117.2763,64.1660,0.9438,1.0329,0.3148
4,48.1934,6253.5240,79.0792,0.9204,1.1881,0.5614
Mean,42.7052,5079.5634,71.0687,0.9347,1.3035,0.4182
Std,5.1672,770.1224,5.3665,0.0098,0.2770,0.1018


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [22]:
dni_final = finalize_model(dni_blender)

In [27]:
ghi_top3 = compare_models(sort = 'MSE',n_select=3,
                     exclude=['rf','dt','ada','et','en','lasso','br','ridge','huber','gbr','par','lr','lar'],
                     )

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,8.5662,228.2863,15.0771,0.9978,0.3967,0.0741,11.2340
catboost,CatBoost Regressor,9.1609,237.0029,15.3630,0.9977,0.6316,0.0850,13.1880
xgboost,Extreme Gradient Boosting,10.5222,287.6386,16.8122,0.9973,0.8793,0.0908,7.6240
knn,K Neighbors Regressor,11.5032,472.2953,21.6875,0.9955,0.1447,0.0951,6.5560
omp,Orthogonal Matching Pursuit,66.9566,26455.1572,123.2699,0.7628,2.6930,0.6651,7.2780
llar,Lasso Least Angle Regression,291.8811,108688.8831,328.5601,-0.0185,4.0042,2.5320,8.3880
dummy,Dummy Regressor,291.8811,108688.8831,328.5601,-0.0185,4.0042,2.5320,5.2100


Processing:   0%|          | 0/35 [00:00<?, ?it/s]

In [28]:
ghi_blender = blend_models(ghi_top3)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,10.6252,267.3025,16.3494,0.9977,0.9492,0.0860
1,8.2547,216.3487,14.7088,0.9975,0.5841,0.0677
2,7.9274,184.4978,13.5830,0.9985,0.5021,0.0640
3,7.2589,185.1566,13.6072,0.9978,0.3168,0.0650
4,9.5536,254.7492,15.9609,0.9979,0.4839,0.0835
Mean,8.7240,221.6109,14.8419,0.9979,0.5672,0.0732
Std,1.2086,34.4089,1.1534,0.0004,0.2098,0.0095


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [29]:
ghi_final = finalize_model(ghi_blender)

### Apply pre-process function to test data

In [31]:
test=pre_process(test)

In [37]:
res=pd.DataFrame(columns=['Clearsky DHI','Clearsky DNI','Clearsky GHI'])
res['Clearsky DHI']=predict_model(dhi_final, data=test.loc[:,X.columns])['prediction_label']
res['Clearsky DNI']=predict_model(dni_final, data=test.loc[:,X.columns])['prediction_label']
res['Clearsky GHI']=predict_model(ghi_final, data=test.loc[:,X.columns])['prediction_label']

for col in ['Clearsky DHI','Clearsky DNI','Clearsky GHI']:
    res[col]=res[col].apply(lambda x:0 if x<0 else x)
    
    
res=res.round(2)

In [39]:
res.head()

,Clearsky DHI,Clearsky DNI,Clearsky GHI
0,0.0,3.20,0.0
1,0.0,2.95,0.0
2,0.0,2.95,0.0
3,0.0,2.35,0.0
4,0.0,2.40,0.0


In [ ]:
res.to_csv("pycaret_blend_1.csv",index=False)